# FIRST STEP: AVOID SUPERPOSITION

In [3]:
#Mounting Google Drive files
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Unzipping the Dataset Required

import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/FinalDataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [9]:
#Unzipping the Dataset Required


with zipfile.ZipFile('/content/drive/MyDrive/Correct Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [5]:
import os 
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


#Saving the imgs names
path = Path('/content/FinalDataset/')
RetImgs = [i  for i in os.listdir(path/'RET'/'Img')]

In [7]:
#Adding common names in folders
Names = []
for i in RetImgs:
  #if (i in EPImgs) & (i in EP_PRImgs) & (i in PRImgs):
    Names.append(i)

In [ ]:
#Creating necessary directories
import os
os.mkdir(f'/content/CorrectDataset')
LayersFolders = ['EP', 'EP_PR', 'PR','RET','REST_RET', 'Original']
for i in LayersFolders: 
  os.mkdir(f'/content/CorrectDataset/{i}')

In [ ]:
#Modifying images to avoid superposition
import cv2
RetinLayers = ['EP', 'EP_PR', 'PR','RET']

photo = []
pixels = []

for i in Names:
  for j in RetinLayers: 
    photo.append(plt.imread(path/j/'masks'/i))
  IMAGEN_SIN_EP = (photo[3]-photo[0])
  cell_prev = 0
  for column in range(IMAGEN_SIN_EP.shape[1]):
    for row in range(IMAGEN_SIN_EP.shape[0]): 
      if IMAGEN_SIN_EP[row,column]==0 and cell_prev==1: 
        IMAGEN_SIN_EP[row:,column] = 0
      cell_prev = IMAGEN_SIN_EP[row,column]
  EP = photo[3]-IMAGEN_SIN_EP
  IMAGEN_SIN_EP_EP_PR = IMAGEN_SIN_EP-photo[1]
  EP_PR = photo[1]
  PR = photo[2]
  RESTO_RET = IMAGEN_SIN_EP_EP_PR-photo[2]
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[0],i),EP*255)
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[1],i), EP_PR*255)
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[2],i), PR*255)
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[3],i), photo[3]*255)
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[4],i), RESTO_RET*255)
  cv2.imwrite(os.path.join('/content/CorrectDataset',LayersFolders[5], i), plt.imread(path/'EP'/'Img'/i)*255)
  photo = []

In [ ]:
#Creating zip file with obtained images
import shutil
shutil.make_archive('Correct Dataset', 'zip', '/content/CorrectDataset')

# SECOND STEP: OBTAIN COLOURED AND RESIZED MASKS

In [10]:
#Saving names
import os
LayersFolders = ['EP', 'EP_PR', 'PR','RET','REST_RET', 'Original']
path = Path('/content/Correct Dataset')
RetImgs = [i  for i in os.listdir(path/'RET')]
Names = []
for i in RetImgs:
    Names.append(i)

In [2]:
#Comprobacion de que las imagenes estan bien
for name in Names: 
   
    EP_PR = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[1],name))
    PR = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[2],name))
    RET = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[3],name))
    REST_RET = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[4],name))
    Original = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[5],name)) 
    EP = plt.imread(os.path.join('/content/Correct Dataset',LayersFolders[0],name))
    e= EP+EP_PR+PR+REST_RET
    e= np.sum(e)
    h = np.sum(RET)
    if e != h:
         print(e, name, 'Has superposition')


NameError: ignored

In [11]:
#Cojo mascaras del Dataset tratado y las añado en una unica imagen con colores por capa
def color_mask(path,name):
  Layers_Colour = ['/EP/', '/EP_PR/', '/PR/', '/REST_RET/']
  MaskLayer = []
  for j in Layers_Colour: 
    MaskLayer.append(plt.imread(path + j + name))
  MaskColor = MaskLayer[0] + MaskLayer[1]*2 + MaskLayer[2]*3 + MaskLayer[3]*4
  return MaskColor

In [12]:
#Defining image treatment to avoid losing quality
def Treatment(img,size): 
  image = cv2.resize(img,size)
  for i in range(10):
    for i in range(image.shape[1]): 
      for j in range(image.shape[0]): 
        try:
          if (image[j,i]!=1) and (image[j,i]!=2) and (image[j,i]!=3) and (image[j,i]!=4) and (image[j,i]!=0):
            image[j,i] = image[j+1,i]
          if (image[j,i]!=image[j-1,i]) and ((image[j,i]!=image[j+1,i])):
            image[j,i] = image[j+1,i]
        except:
          continue
  return image

In [13]:
#Aceleracion por GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
#Installing GPU libraries for Numpy arrays
!pip install minpy
!pip install --upgrade mxnet


     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for minpy: filename=minpy-0.3.4-py3-none-any.whl size=76355 sha256=6cc04b6ca7e1f60920fa66ed47db33e30b47f6de620958bb9938d89dbdbb1382
  Stored in directory: /root/.cache/pip/wheels/00/7a/24/982da0b566f63edac04a827abf2d669de2ed67549e5d27767c
Successfully built minpy
     |████████████████████████████████| 46.9 MB 46 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
#Obtaining treated images and saving them in a folder
path = Path('/content/Correct Dataset')
from minpy.context import gpu
import cv2
j=0
with gpu(0):
  for i in Names: 
    j+=1
    MaskColor = color_mask('/content/Correct Dataset',i)
    MaskColor = Treatment(MaskColor,(512,512))
    print(j)
    cv2.imwrite(os.path.join('/content/ColouredMasks',i), MaskColor)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


In [ ]:
#Creating zip folder with treated images
shutil.make_archive('ColouredMasks', 'zip', '/content/ColouredMasks')

In [ ]:
plt.figure(figsize=(100,100))
hola = plt.imread('/content/ColouredMasks/0.png')
plt.imshow(hola)